In [ ]:
#| default_exp helpers

# helpers 


In [1]:
#| export

from tombo.tombo_stats import PerReadStats, TomboStats
from tombo.tombo_helper import intervalData

import pandas as pd
import numpy as np

In [ ]:
# | export


def tombo_read_stats(tombo_stats, contig, start, end,fill=False):
    data = pd.DataFrame(
        PerReadStats(tombo_stats).get_region_per_read_stats(
            intervalData(chrm=contig, start=start, end=end, strand='+'))).pivot(
                index='read_id', columns=['pos', ], values='stat')
    
    data.columns = data.columns.astype(int)
    data.columns = [i+1 for i in data.columns]
    if fill:
        data = data.reindex(range(start+1, end+1), axis=1)
    data = -np.log(data)
    return data


In [ ]:
#| export

def baleen_read_stats(baleen_stats, contig, start, end, stat=None,fill=False):
    data = pd.read_csv(baleen_stats, index_col=0)
    if data.max().max() > 1:
        data = data/5
    data = data.rolling(3, axis=1, center=True).sum()/3
    data.columns = data.columns.astype(int)
    if fill:
        data = data.reindex(range(start+1, end+1), axis=1,fill_value=None)
    # else:
    #     data = data.reindex(data.columns, axis=1)
    return data

In [ ]:
#| export 

# mod_list = [0,0,1,0.5,0.6,0.6,...] representing the posibility of modification for each read on the locus
def _modtect(mod_list):
    mod_n = (mod_list > 0.5).sum()
    total = len(mod_list)
    f = mod_n/total
    lmf = 1
    lm0 = 1
    for i in mod_list:
        lmf *= _posibility(i,f)
        lm0 *= _posibility(i,0)
    return np.log10(lmf/lm0)


def _posibility(mod_p,f):
    return f*mod_p + (1-f)*(1-mod_p)

In [ ]:
#| export

def _check_baleen(baleen_stats, contig, start, end,fill=False):
    if isinstance(baleen_stats, str):
        baleen_stats = baleen_read_stats(baleen_stats, contig, start, end,fill=False)
    return baleen_stats


def baleen_transcript_stats(baleen_stats, contig, start, end, window=2,fill=False, **kwargs):
    baleen_stats = _check_baleen(baleen_stats, contig, start, end,fill)
    baleen_stats[baleen_stats < 0.2] = -0.1
    data = pd.DataFrame(baleen_stats.rolling(window, axis=1, center=True).sum(
    ).fillna(0).sum(axis=0, skipna=True), columns=['stat'])
    data['pos'] = data.index.astype(int)
    return data


def baleen_transcript_modtect_stats(baleen_stats, contig, start, end,):
    baleen_stats = _check_baleen(baleen_stats, contig, start, end)
    data = pd.DataFrame(baleen_stats.apply(_modtect, axis=0), columns=['stat'])
    return data


In [ ]:
#| export

def tombo_transcript_stats(tombo_stats, contig, start, end, stat=None,fill=False):
    ts = TomboStats(tombo_stats)
    if ts.is_model_stats:
        data = pd.DataFrame(
            ts.get_reg_stats(contig, '+', start, end)).melt(
                id_vars='pos', value_vars='frac', var_name='type',value_name='stat')
    else:
        data = pd.DataFrame(ts.get_reg_stats(contig, '+', start, end)) 
    data['stat'] += 0.001
    data['stat'] = -np.log10(data['stat'])
    data['pos'] = data['pos'] + 1
    if fill:
        data.index = data['pos']
        data = data.reindex(range(start+1, end+1), axis=0, fill_value=0)
        data['pos'] = data.index.astype(int)
    return data




In [ ]:
#| export

def xpore_transcript_stats(xpore_stats, contig, start, end, stat=None,fill=False):
    if stat == 'pvalue':
        data = pd.read_csv(xpore_stats).iloc[:, [0, 1, 4]]
        # if I used np.log1p, I may encounter FloatingPointError: underflow encountered in log1p
        # when I check the data, I found that the pvalue is very near to 0, 1.22e-300 something like that.
        data.iloc[:, 2] += 0.001
        data.iloc[:, 2] = -np.log(data.iloc[:, 2])
    else:
        data = pd.read_csv(xpore_stats).iloc[:, [0, 1, 5]]
    select = (data['id'] == contig) & (
        data['position'] <= end) & (data['position'] >= start)
    data = data.loc[select,:]
    data.columns = ['contig', 'pos', 'stat']
    data['pos'] = data['pos'] + 1
    if fill:
        data.index = data['pos']
        data = data.reindex(range(start+1, end+1), axis=0,fill_value=0)
        data['pos'] = data.index.astype(int)
    return data
